In [73]:
## imports here !!!
import numpy as np
import pandas as pd
import xgboost as xgb


from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# Trial 1

In [74]:
'''
Importing data
'''
train_data= pd.read_csv('iith_foml_2020_train.csv')
train = pd.DataFrame(train_data)
train.head()


,Feature 1 (Discrete),Feature 2 (Discrete),Feature 3 (Discrete),Feature 4 (Discrete),Feature 5 (Discrete),Feature 6 (Discrete),Feature 7 (Discrete),Feature 8 (Discrete),Feature 9,Feature 10,...,Feature 16,Feature 17,Feature 18,Feature 19 (Discrete),Feature 20 (Discrete),Feature 21 (Discrete),Feature 22 (Discrete),Feature 23 (Discrete),Feature 24,Target Variable (Discrete)
0,1404,12,64,14,3,1,1,1,110.502,35775.2,...,NaN,NaN,15.04,104,12,2,32,1409,37677.1,1
1,909,0,235,32,1,1,1,1,-40.448,35779.4,...,2200.3,4900.005,12.03,20,1,0,13,909,25239.1,1
2,654,3,175,2,1,1,1,1,-27.445,35770.4,...,1973.3,10000.004,13.01,1,1,0,13,654,27683.5,1
3,1372,12,382,14,2,0,1,0,0.001,509.2,...,NaN,NaN,NaN,313,12,10,54,1377,39363.2,0
4,786,3,199,2,1,0,1,0,0.001,612.1,...,NaN,NaN,NaN,171,1,5,11,786,40044.4,2


In [75]:
'''
Dealing with missing values/....
1. Dropping them
'''

train = train[train.columns[~train.isnull().any()]]
train.head()


,Feature 1 (Discrete),Feature 2 (Discrete),Feature 3 (Discrete),Feature 4 (Discrete),Feature 5 (Discrete),Feature 6 (Discrete),Feature 7 (Discrete),Feature 8 (Discrete),Feature 19 (Discrete),Feature 20 (Discrete),Feature 21 (Discrete),Feature 22 (Discrete),Feature 23 (Discrete),Target Variable (Discrete)
0,1404,12,64,14,3,1,1,1,104,12,2,32,1409,1
1,909,0,235,32,1,1,1,1,20,1,0,13,909,1
2,654,3,175,2,1,1,1,1,1,1,0,13,654,1
3,1372,12,382,14,2,0,1,0,313,12,10,54,1377,0
4,786,3,199,2,1,0,1,0,171,1,5,11,786,2


In [76]:
'''
Data Analysis
'''
train.groupby("Target Variable (Discrete)")["Target Variable (Discrete)"].count()

Target Variable (Discrete)
0     249
1     488
2     109
3       3
4       3
5      41
6      70
7       5
8       7
9       2
10      1
11      1
12      1
13      3
14      5
15      4
16      1
17      1
Name: Target Variable (Discrete), dtype: int64

In [77]:
'''
Dividing into X, Y
'''
Y = train['Target Variable (Discrete)']
X = train.drop('Target Variable (Discrete)',axis = 1)



In [81]:
'''
Training with stratified K-fold
'''

i=1
kf = StratifiedKFold(n_splits=5,random_state=1,shuffle=True)

param_grid = [{'min_child_weight': np.arange(0.1, 10.1, 2)}] #set of trial values for min_child_weight


for train_index,test_index in kf.split(X,Y):
    print('\n{} of kfold {}'.format(i,kf.n_splits))
    xtr,xvl = X.loc[train_index],X.loc[test_index]
    ytr,yvl = Y[train_index],Y[test_index]

    model =GridSearchCV(XGBClassifier(), param_grid, cv=5, scoring= 'f1_macro',iid=True)
    model.fit(xtr, ytr)

    print (model.best_params_)
    y_pred=model.predict(xvl)
    y_pred = [round(value) for value in y_pred]
    
    
    print('accuracy_score',accuracy_score(yvl,y_pred) * 100)
    i+=1
        
        


/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)



1 of kfold 5


/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'min_child_weight': 0.1}
accuracy_score 90.95477386934674

2 of kfold 5


/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


...

{'min_child_weight': 2.1}
accuracy_score 90.45226130653266

4 of kfold 5


/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'min_child_weight': 2.1}
accuracy_score 85.92964824120602

5 of kfold 5


/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'min_child_weight': 2.1}
accuracy_score 88.88888888888889


In [104]:
'''
Generating the test samples...
'''
test_data= pd.read_csv('iith_foml_2020_test.csv')
test = pd.DataFrame(test_data)
test = test[test.columns[~test.isnull().any()]]
test = test[X.columns]

predictions = model.predict(test)

print(predictions.size)

output_test_data = pd.DataFrame() 
output_test_data['Category'] = predictions

output_test_data['Id'] = list(np.arange(1,predictions.size+1))

submission = output_test_data[['Id','Category']]
submission.to_csv("S_001.csv", index=False)
submission.head()




def predict_and_submission(test_data,model,title):### test_data in pd format
    test = pd.DataFrame(test_data)
    predictions = model.predict(test)

    output_test_data = pd.DataFrame() 
    output_test_data['Category'] = predictions

    output_test_data['Id'] = list(np.arange(1,predictions.size+1))

    submission = output_test_data[['Id','Category']]
    submission.to_csv(title, index=False)
    submission.head()
    


426


,Id,Category
0,1,6
1,2,2
2,3,1
3,4,1
4,5,1


In [94]:
test.head()

,Feature 1 (Discrete),Feature 2 (Discrete),Feature 3 (Discrete),Feature 4 (Discrete),Feature 5 (Discrete),Feature 6 (Discrete),Feature 7 (Discrete),Feature 8 (Discrete),Feature 19 (Discrete),Feature 20 (Discrete),Feature 21 (Discrete),Feature 22 (Discrete),Feature 23 (Discrete)
0,146,12,42,14,7,1,1,1,44,12,2,42,146
1,35,0,12,5,0,0,1,0,17,6,8,0,35
2,1018,8,259,2,1,1,1,1,1,1,3,20,1018
3,383,7,117,5,1,1,1,1,101,6,3,20,383
4,1216,7,40,5,2,0,1,4,276,6,7,43,1221
